# Sentinel-2 Super-Resolution (4×)

**Upscale Sentinel-2 imagery from 10m/pixel → 2.5m/pixel using SwinIR**

Make sure GPU is enabled: `Runtime → Change runtime type → GPU`

In [ ]:
# Clone repository
!git clone https://github.com/Rishikarnatakam/Klymo.git
%cd Klymo

# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Verify GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Google Earth Engine authentication
import ee

# Authenticate (opens popup - sign in with Google)
ee.Authenticate()

# Initialize with your project
ee.Initialize(project='klymo-486313')

print("✓ GEE authenticated")

In [ ]:
# Fetch real Sentinel-2 tile from Delhi (streamed via GEE API)
from src.data.gee_fetcher import GEEFetcher

# Already authenticated above
fetcher = GEEFetcher(authenticate=False)
fetcher.authenticated = True
fetcher.ee = ee

delhi_tile = fetcher.fetch_tile('delhi', tile_size=256)
print(f"✓ Fetched tile: {delhi_tile.shape}")

# Show the tile
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 6))
plt.imshow(delhi_tile)
plt.title('Sentinel-2 Delhi (10m/pixel)')
plt.axis('off')
plt.show()

In [ ]:
# Run super-resolution
from src.inference.pipeline import SuperResolutionPipeline

pipeline = SuperResolutionPipeline(device=DEVICE)
results = pipeline.run(delhi_tile)

print(f"Input:  {results['lr'].shape} (10m/pixel)")
print(f"Output: {results['sr'].shape} (2.5m/pixel)")

In [ ]:
# Visualize comparison (LR → Bicubic → SwinIR)
import matplotlib.pyplot as plt
from src.data.preprocessing import to_8bit_visualization
from skimage.transform import resize

lr = results['lr']
bicubic = results['bicubic']
sr = results['sr']

lr_up = resize(lr, sr.shape[:2], order=0, preserve_range=True)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
axes[0].imshow(to_8bit_visualization(lr_up))
axes[0].set_title('LR Input (10m/pixel)', fontsize=14)
axes[0].axis('off')

axes[1].imshow(to_8bit_visualization(bicubic))
axes[1].set_title('Bicubic 4× (Baseline)', fontsize=14)
axes[1].axis('off')

axes[2].imshow(to_8bit_visualization(sr))
axes[2].set_title('SwinIR 4× (2.5m/pixel)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.savefig('outputs/visualizations/comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved to outputs/visualizations/comparison.png")

In [ ]:
# Compute metrics
from src.metrics.psnr import compute_psnr
from src.metrics.ssim import compute_ssim

psnr = compute_psnr(bicubic, sr)
ssim = compute_ssim(bicubic, sr)

print(f"Metrics (SwinIR vs Bicubic):")
print(f"  PSNR: {psnr:.2f} dB")
print(f"  SSIM: {ssim:.4f}")

In [ ]:
# Download results
from google.colab import files
files.download('outputs/visualizations/comparison.png')